# Entrenament del model

In [ ]:
import pandas as pd
import joblib, os, shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime

In [36]:
data = pd.read_csv("../res/dataset_final.csv")
data = data.set_index("_id")
data

,ano,almacenamiento,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
_id,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,0.0,4.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,1.2,12.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00


In [37]:
data.dropna(inplace=True)

## Transformació de valors

### Columnes de text

In [38]:
def aplicar_onehotencoder(df, columnas_object = None):
    if not columnas_object: columnas_object = df.select_dtypes(include=['object']).columns
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded_data = encoder.fit_transform(df[columnas_object])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columnas_object))
    encoded_df.index = df.index
    return encoded_df, encoder

df_onehot, encoder_ohe = aplicar_onehotencoder(data)
df_onehot

,marca_8849pro,marca_akozon,marca_alcatel,marca_apple,marca_artfone,marca_asus,marca_azuiqimeng,marca_bewinner,marca_blackview,marca_ciciglow,...,pantalla_tipo_dlp,pantalla_tipo_fhd+,pantalla_tipo_hd,pantalla_tipo_hd+,pantalla_tipo_ips,pantalla_tipo_lcd,pantalla_tipo_led,pantalla_tipo_oled,pantalla_tipo_superamoled,pantalla_tipo_xdr
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00J8OA220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00JC8MD7Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
B00TKALUDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B00TUXHZTW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
B09QH71RZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0DK4NBZWB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Columnes numeriques

In [39]:
def aplicar_standardscaler(df, columnas_numericas=None):
    if columnas_numericas is None:
        columnas_numericas = df.select_dtypes(include=['number']).columns
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columnas_numericas])
    scaled_df = pd.DataFrame(scaled_data, columns=columnas_numericas, index=df.index)
    return scaled_df, scaler

df_scaled, scaler = aplicar_standardscaler(data)
print(data.select_dtypes(include=['number']).columns)

Index(['ano', 'almacenamiento', 'pantalla_in', 'velocidad_cpu_ghz', 'ram',
       'grosor', 'peso', 'ancho_px', 'alto_px', 'bateria',
       'promedio_valoraciones', 'precio_anterior', 'precio_actual'],
      dtype='object')


### Unim les columnes

In [40]:
df_onehot.index = df_scaled.index
df_normalitzat = pd.concat([df_onehot, df_scaled], axis=1)
df_normalitzat

,marca_8849pro,marca_akozon,marca_alcatel,marca_apple,marca_artfone,marca_asus,marca_azuiqimeng,marca_bewinner,marca_blackview,marca_ciciglow,...,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.086451,-1.185651,-0.301268,-1.810500,-1.305174,-1.279807,0.022950,0.137299,-0.260530,0.308796
B00J8OA220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.069004,-1.045971,1.837201,0.647642,-1.859876,-2.194315,-1.737177,-0.280502,-0.260530,4.795794
B00JC8MD7Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.079181,-0.487252,-0.421542,-0.770772,-0.306709,0.777834,-0.158882,2.359833,0.194863,0.536993
B00TKALUDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.111168,-0.766611,-0.173777,-1.832031,-0.306709,0.777834,-1.504433,-0.028958,-1.137162,-1.015195
B00TUXHZTW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.093721,0.350828,-0.149723,-1.842236,4.796555,0.434894,-1.504433,-0.170751,-1.121906,-0.971629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.210156,-0.766611,0.011444,-0.001155,1.246458,-1.108337,0.022950,-0.279149,-0.260530,-0.637518
B09QH71RZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.082089,-0.487252,-0.411920,-0.770772,1.246458,-1.108337,0.022950,5.448437,-0.457487,-0.460777
B0DK4NBZWB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.061734,-0.207892,0.011444,-0.001155,-0.306709,0.777834,0.022950,-0.278265,-0.260530,0.660973


In [41]:
# Separar features y target
X = df_normalitzat.drop(columns=["precio_actual"])
y = df_normalitzat["precio_actual"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## MODELS

In [42]:
# Entrenar modelo GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
history_gb = gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)


# Entrenar modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
history_rf = rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


### Metriques

In [ ]:
# Evaluar model GradientBoostingRegressor
gb_results = {
    "MAE": mean_absolute_error(y_test, y_pred_gb),
    "MSE": mean_squared_error(y_test, y_pred_gb),
    "RMSE:": mean_squared_error(y_test, y_pred_gb) ** 0.5,
    "R2 Score": r2_score(y_test, y_pred_gb),
}

# Evaluar model RandomForestRegressor
rf_results = {
    "MAE": mean_absolute_error(y_test, y_pred_rf),
    "MSE": mean_squared_error(y_test, y_pred_rf),
    "RMSE:": mean_squared_error(y_test, y_pred_rf) ** 0.5,
    "R2 Score": r2_score(y_test, y_pred_rf),
}

# Mostrar resultados
print("GradientBoostingRegressor:")
for metric, value in gb_results.items():
    print(f"  {metric}: {value}")

print("\nRandomForestRegressor:")
for metric, value in rf_results.items():
    print(f"  {metric}: {value}")

GradientBoostingRegressor:
  MAE: 0.2087816542252643
  MSE: 0.08361847786602263
  RMSE:: 0.28916859764853897
  R2 Score: 0.9146139998282622

RandomForestRegressor:
  MAE: 0.17316832740844282
  MSE: 0.07251681346446154
  RMSE:: 0.2692894603664643
  R2 Score: 0.9259503305375713


### Ajustem el millor model

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

rf_model_ajustat = RandomForestRegressor(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300, 500, 1000],  
    'max_depth': [10, 20, 30, None],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],  
    'max_features': ['sqrt', None]
}

# Configuración de RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf_model_ajustat,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)
best_rf = random_search.best_estimator_
y_pred_rf_ajustat = best_rf.predict(X_test)

rf_results_ajustat = {
    "MAE": mean_absolute_error(y_test, y_pred_rf_ajustat),
    "MSE": mean_squared_error(y_test, y_pred_rf_ajustat),
    "RMSE:": mean_squared_error(y_test, y_pred_rf_ajustat) ** 0.5,
    "R2 Score": r2_score(y_test, y_pred_rf_ajustat),
}

In [45]:
print("\nRandomForestRegressor Ajustat:")
for metric, value in rf_results_ajustat.items():
    print(f"  {metric}: {value}")
print(f"Hiperparametres: {random_search.best_params_}")


RandomForestRegressor Ajustat:
  MAE: 0.1720115495195531
  MSE: 0.07174897805204349
  RMSE:: 0.267859997110512
  R2 Score: 0.9267343964082947
Hiperparametres: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20}


Podem vore que el n_estimators esta en un valor extrem, per tant seria convenient vorer realment si la millora es significariva respecte a valors mes baixos

In [52]:
import time

hiperparametres = random_search.best_params_
hiperparametres['n_estimators'] = None
for n in [20, 50, 100, 500, 1000, 2000]:
    hiperparametres['n_estimators'] = n
    rf = RandomForestRegressor(**hiperparametres, random_state=42)
    start = time.time()
    rf.fit(X_train, y_train)
    end = time.time()
    
    y_pred = rf.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    print(f"n_estimators={n} | R2: {r2:.5f} | Tiempo: {end - start:.2f} s")


n_estimators=20 | R2: 0.92750 | Tiempo: 0.15 s
n_estimators=50 | R2: 0.92546 | Tiempo: 0.38 s
n_estimators=100 | R2: 0.92549 | Tiempo: 0.79 s
n_estimators=500 | R2: 0.92675 | Tiempo: 4.18 s


KeyboardInterrupt: 

Com podem vore la millora es practicament insignificant i sols estem utilitzant mes rendiment innecesari per tant deixarem el n_estimators amb 500

In [59]:
hiperparametres = random_search.best_params_
hiperparametres['n_estimators'] = 500
rf_model_ajustat = RandomForestRegressor(**hiperparametres, random_state=42)

history_rf = rf_model_ajustat.fit(X_train, y_train)

y_pred_rf_ajustat = rf_model_ajustat.predict(X_test)

rf_results_ajustat = {
    "MAE": mean_absolute_error(y_test, y_pred_rf_ajustat),
    "MSE": mean_squared_error(y_test, y_pred_rf_ajustat),
    "RMSE:": mean_squared_error(y_test, y_pred_rf_ajustat) ** 0.5,
    "R2 Score": r2_score(y_test, y_pred_rf_ajustat),
}

print("\nRandomForestRegressor Ajustat:")
for metric, value in rf_results_ajustat.items():
    print(f"  {metric}: {value}")


RandomForestRegressor Ajustat:
  MAE: 0.17187388553448393
  MSE: 0.07173007077114997
  RMSE:: 0.2678247015701688
  R2 Score: 0.9267537033501433


## GUARDAR MODEL

In [ ]:
import os
import shutil
import joblib
import pandas as pd
from datetime import datetime

def guardar_modelo(modelo, encoder, scaler, X_train, X_test, y_train, y_test, model_type="gb"):
    fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
    folder_name = f"model_{model_type}_{fecha_hora_actual}"
    models_path = os.path.join("./models", folder_name)
    os.makedirs(models_path, exist_ok=True)

    joblib.dump(modelo, os.path.join(models_path, 'model.joblib'))
    joblib.dump(encoder, os.path.join(models_path, 'encoder_ohe.joblib'))
    joblib.dump(scaler, os.path.join(models_path, 'scaler.joblib'))

    X_train.to_csv(os.path.join(models_path, 'X_train.csv'))
    X_test.to_csv(os.path.join(models_path, 'X_test.csv'))
    y_train.to_csv(os.path.join(models_path, 'y_train.csv'))
    y_test.to_csv(os.path.join(models_path, 'y_test.csv'))

    try:
        shutil.make_archive(models_path, 'zip', root_dir=models_path)
        shutil.rmtree(models_path)
        print(f"Carpeta comprimida exitosamente a {folder_name}.zip")
    except Exception as e:
        print(f"Error: {e}")

guardar_modelo(rf_model_ajustat, encoder_ohe, scaler, X_train, X_test, y_train, y_test, model_type="rf")


Carpeta comprimida exitosamente a model_rf_05_03_2025_21_59.zip


## Prediccions

In [ ]:
peticion = {
    "ano":2024,
    "almacenamiento": 128,
    "marca": 'oukitel',
    "pantalla_in": 6, 
    "pantalla_tipo": 'lcd', 
    "velocidad_cpu_ghz": 2,
    "ram": 4, 
    "grosor": 1.54, 
    "peso":300,
    'ancho_px':720, 
    'alto_px':1440,     
    "bateria": 6300,  
    'promedio_valoraciones':13566.95, 
    'precio_anterior':146.99,
    'precio_actual': 0
}

peticion_df = pd.DataFrame([peticion])
model_path = os.path.join('./models', 'model_rf_05_03_2025_21_59')
scaler = joblib.load(os.path.join(model_path, 'scaler.joblib'))
encoder = joblib.load(os.path.join(model_path, 'encoder_ohe.joblib'))
model = joblib.load(os.path.join(model_path, 'model.joblib'))

peticion_scaled = scaler.transform(peticion_df.select_dtypes(include=['number']))
peticion_encoder = encoder.transform(peticion_df.select_dtypes(include=['object']))

peticion_df_scaled = pd.DataFrame(peticion_scaled, columns=peticion_df.select_dtypes(include=['number']).columns)
peticion_df_encoded = pd.DataFrame(peticion_encoder, columns=encoder.get_feature_names_out())

peticion_final = pd.concat([peticion_df_scaled, peticion_df_encoded], axis=1)
peticion_final.drop(['precio_actual'], inplace = True, axis = 1)

X_train = pd.read_csv(os.path.join(model_path, 'X_train.csv'))
X_train.drop(['_id'], inplace = True, axis = 1)
peticion_final = peticion_final[X_train.columns]

prediccio = model.predict(peticion_final)

peticion_scaled_with_pred = peticion_scaled.copy()
peticion_scaled_with_pred[:, peticion_df_scaled.columns.get_loc('precio_actual')] = prediccio
prediccion_escalada_inversa = scaler.inverse_transform(peticion_scaled_with_pred)

prediccion_final = prediccion_escalada_inversa[0, peticion_df_scaled.columns.get_loc('precio_actual')]

print(prediccion_final)

138.56340077793763
